In [ ]:
!pip install apyori

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#CONNECTION TO SQL SERVER

import pyodbc
import pandas as pd
from pandas import DataFrame

server = 'fill'
database = 'fill'
username = 'fill'
password = 'fill'   
driver= '{SQL Server}'

database = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = database.cursor()

#SQL QUERY
query = '''blank  '''
df = pd.read_sql(query, database)
df.head()

In [ ]:
array_list = df.groupby(df.FFA_TestSum_ReqId.values).agg(list).to_dict('records')

In [ ]:
for key in array_list:
   print(key)

In [ ]:
df.info()

In [ ]:
#adjust according to data imported
data = df
data = data[['Result1','Result2','Result3','Result4','Result5']]
data.head()

In [ ]:
data.info()

In [ ]:
dataset = data 
transactions = []
for i in range(0, 213):    #correct to numbers of row from your dataset
  transactions.append([str(dataset.values[i,j]) for j in range(0, 4)])  #choose number of max columns, place max column as a first one 

In [ ]:
from apyori import apriori
rules = apriori(transactions = transactions, min_support = 0.012, min_confidence = 0.2, min_lift =3, min_length=2, max_length=2) #refer to me for metrics adjustments

In [ ]:
results = list(rules)
results

In [ ]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [ ]:
resultsinDataFrame

In [ ]:
resultsinDataFrame.nlargest(n = 10, columns = 'Lift') #grupowanie

In [ ]:

#EXPORT TO SQL

from sqlalchemy import create_engine
import urllib
import pyodbc
import pandas as pd

#SAME CONNECTION
server = 'fill'
database = 'fill'
username = 'fill'
password = 'fill'   
driver= '{SQL Server}'
database = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = database.cursor()


#NEW Way of exporting
#Insert Dataframe into SQL Server:
for index, row in resultsinDataFrame.iterrows():
    #adjust query below! - according to database table structure
    cursor.execute("INSERT INTO NLP_test.TargetTable1 (processed_date, ProductName, [lineNo], review_id, sentiment, key_point) values(?,?,?,?,?,?)", row.processed_date, row.ProductName, row.lineNo, row.review_id, row.sentiment, row.key_point)
database.commit()
cursor.close()

print('execution stopped')